In [21]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
pd.set_option('display.max_columns', None)

In [78]:
#データを整理
df23_24 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_23-24シーズン.csv')
df24_25 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_24-25シーズン.csv')
df_box = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】ボックススコア.csv')
df_game = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】試合データ.csv')
df_player = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】選手マスタ.csv')


# プレイバイプレイ結合
df_pbp = pd.concat([df23_24, df24_25], ignore_index=True)

# プレイタイムを正規化（全角コロン、DNPなどを処理）
df_box['プレイタイム_秒'] = (
    df_box['プレイタイム']
    .astype(str)                               # 念のため文字列化
    .str.replace('：', ':', regex=False)       # 全角コロン対策
    .apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]) if ':' in x else 0)
)

# 試合ID、チームIDごとにデータを整理
df_box_gameid = df_box.drop(columns=['ホームアウェイ','チーム名','チーム名英','選手ID','背番号','選手名','スターティングフラグ'])
df_box_gameid_sum = df_box_gameid[df_box_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()

# 試合IDごとに勝敗を割り当て
# 1. 試合IDごとに得点の最大値を取得
max_scores = df_box_gameid_sum.groupby('試合ID')['得点'].transform('max')

# 2. 自チームの得点が最大なら1（勝ち）、そうでなければ0（負け）
df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

# 選手のデータを集計
# 前処理

df_box = df_box.drop(['背番号','チームID','ホームアウェイ','スターティングフラグ'], axis=1, errors='ignore')

df_box_period18 = df_box[df_box['ピリオド区分'] == 18].groupby(['選手ID']).sum(numeric_only=True).reset_index()

# 数値列を取得（選手ID 等のキー列は除外）
num_cols = df_box_period18.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c != 'プレイタイム_秒']  # プレイタイム自身は除外

# 必要なら選手ID が数値になっている場合は除外
if '選手ID' in num_cols:
    num_cols.remove('選手ID')

# 割り算を行い新しいデータフレームを作成
df_box_period18_persec = df_box_period18.copy()
df_box_period18_persec[num_cols] = df_box_period18[num_cols].div(df_box_period18['プレイタイム_秒'], axis=0) * 60 * 40

df_player_stats  = df_box_period18_persec.merge(df_player, on='選手ID', how='left').drop(['試合ID','ピリオド区分'], axis=1, errors='ignore')

In [85]:
df_player_stats.loc[
    (df_player_stats['チームID'] == 745) | (df_player_stats['ポジション'] == 'C')
]

,選手ID,得点,3P成功,3P試投,2P成功,2P試投,ダンク,フリースロー成功,フリースロー試投,ファウル,被ファウル,オフェンスリバウンド,ディフェンスリバウンド,トータルリバウンド,ターンオーバ,アシスト,スティール,ブロックショット,被ブロックショット,ファストブレイクポイント,2Pインサイドポイント,セカンドチャンスポイント,プレイタイム_秒,カップID,チームID,背番号,選手名,ポジション,国籍,生年月日,身長,体重,在籍フラグ
62,8490,22.532300,2.273902,5.788114,5.994832,12.137320,0.029531,3.720930,4.370616,1.594684,3.750461,2.126246,10.247324,12.373570,3.277962,5.345146,1.358435,0.295312,0.531561,1.919528,9.922481,3.632337,81270,500,727,22,ニック・ファジーカス,C,JAPAN,19850617,207,114,1
146,8565,8.354430,0.000000,0.000000,3.493671,5.620253,0.000000,1.367089,2.582278,5.924051,1.670886,2.582278,4.101266,6.683544,2.582278,1.518987,0.455696,0.303797,0.151899,0.000000,5.164557,0.911392,15800,500,697,8,太田 敦也,C,JAPAN,19840604,206,113,1
147,8565,8.354430,0.000000,0.000000,3.493671,5.620253,0.000000,1.367089,2.582278,5.924051,1.670886,2.582278,4.101266,6.683544,2.582278,1.518987,0.455696,0.303797,0.151899,0.000000,5.164557,0.911392,15800,503,697,8,太田 敦也,C,JAPAN,19840604,206,113,1
148,8565,8.354430,0.000000,0.000000,3.493671,5.620253,0.000000,1.367089,2.582278,5.924051,1.670886,2.582278,4.101266,6.683544,2.582278,1.518987,0.455696,0.303797,0.151899,0.000000,5.164557,0.911392,15800,507,697,8,太田 敦也,C,JAPAN,19840604,206,113,1
149,8565,8.354430,0.000000,0.000000,3.493671,5.620253,0.000000,1.367089,2.582278,5.924051,1.670886,2.582278,4.101266,6.683544,2.582278,1.518987,0.455696,0.303797,0.151899,0.000000,5.164557,0.911392,15800,512,697,8,太田 敦也,C,JAPAN,19840604,206,113,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,5100000015,11.713646,0.072084,0.288336,4.252955,7.496734,0.324378,2.991485,4.901710,5.370256,3.964619,3.387947,4.721501,8.109448,1.730015,0.937092,0.756882,1.441680,1.009176,1.477722,8.289657,2.270645,66589,504,698,99,川真田 紘也,C,JAPAN,19980616,204,110,1
1516,5100000015,11.713646,0.072084,0.288336,4.252955,7.496734,0.324378,2.991485,4.901710,5.370256,3.964619,3.387947,4.721501,8.109448,1.730015,0.937092,0.756882,1.441680,1.009176,1.477722,8.289657,2.270645,66589,507,2488,99,川真田 紘也,C,JAPAN,19980616,204,110,1
1524,5100000021,11.806316,2.101358,5.972281,2.488451,5.363994,0.000000,0.525340,0.718886,4.064469,0.995380,1.133627,2.737296,3.870923,2.101358,2.377853,1.216576,0.110598,0.525340,1.824864,4.645108,1.023030,86801,501,745,18,星川 堅信,SF,JAPAN,20011101,190,92,1
1525,5100000021,11.806316,2.101358,5.972281,2.488451,5.363994,0.000000,0.525340,0.718886,4.064469,0.995380,1.133627,2.737296,3.870923,2.101358,2.377853,1.216576,0.110598,0.525340,1.824864,4.645108,1.023030,86801,504,745,18,星川 堅信,SF,JAPAN,20011101,190,92,1
